In [126]:
cd "/home/sage/Documents/works/12. Developments/07. sage/02. IdIso/"

/home/sage/Desktop/IdIso-main/IdIso-main/02. IdIso


#  <font color="green">Implementation of IdealToIsogeny algorithm</font>

## <font color="green">0. Prime Search</font>

## <font color="green">1. Prerequisites</font>

### Load required libraries

In [127]:
load('quaternion_tools.sage')
load('isogeny.sage')
load('richelot.sage')
load('const.sage')

### Cornaccia algorithm

In [128]:
def cornacchia(d, m):
    try: 
        (x,y) = DiagonalQuadraticForm(QQ, [1,d]).solve(m)
        if not x.is_integer() or not y.is_integer() : raise Exception("No solution")
        return x, y
    except: raise Exception("No solution")

## <font color="green">2. Setting the lab environment</font>

### 1. Get a random ideal $I$ of norm $\ell^e$

In [129]:
I = GetRandomIdealElle()
assert(I.norm() == ell^6)

### 2. Extract the first ideal factor $I_1$
+ $I = I_1 \cdot I_2 \cdot ... \cdot I_k$ and $n(I_i) = \ell$

In [130]:
ellO0 = QA.ideal(O0.basis()).scale(ell)
I1 = AddIdeal(I, ellO0)

### 3. Compute corresponding isogeny $\phi_1$ to $I_1$
+ $ker(\phi_1) = ker(I_1)$

In [131]:
load('isogeny.sage')
phi1_kernel_generator = left_O0_ideal_to_isogeny_kernel(I1)
phi1 = Isogeny(Fp(0), phi1_kernel_generator, 2^e1*3^e2*ell)
assert(I1.norm() == ell)

### 4. Get the codomain curve $E_1$ of $\phi_1$ and the dual isogeny $\hat{\phi}_1$

In [132]:
E1 = phi1.codomain_curve
phi1_dual = phi1.dual_isogeny()

In [133]:
cofactor = phi1_dual.domain_P.order() // 3^16
T = cofactor * phi1_dual.codomain_P
ellT = phi1.eval(T)
if ellT != phi1.degree * cofactor * phi1_dual.domain_P:
    phi1_dual.codomain_P *= -1
    phi1_dual.codomain_Q *= -1
    assert -ellT == phi1.degree * cofactor * phi1_dual.domain_P
else: assert ellT == phi1.degree * cofactor * phi1_dual.domain_P

T = cofactor * phi1.codomain_P
ellT = phi1_dual.eval(T)
assert ellT == phi1_dual.degree * cofactor * phi1.domain_P

### 5. Compute the right order $\mathcal{O}_1$ of $I_1$
+ $\mathcal{O}_1 \cong End(E_1)$

In [134]:
O1 = I1.right_order()

### 6. Extract the second ideal factor $I_2$
+ $\bar{I_1} I \cap \ell \mathcal{O}_1 = I_2$

In [135]:
I2 = I1.conjugate() * I
I2 = I2.scale(1/ell)
ellO1 = QA.ideal(O1.basis()).scale(ell)
I2 = AddIdeal(I2, ellO1)

assert(I2.norm() == ell and I2.left_order() == O1)

## <font color="green">3. Experiments</font>

+ Compute the generator $\alpha_2$ of $I_2$ such that $I_2 = \alpha_2 \mathcal{O_1} + \ell \mathcal{O_1}$
+ Hopefully $n(\alpha_2)$ is as small as possible
+ Note that $\alpha_2 \in I_2 \subset \mathcal{O}_1$

In [136]:
I2_generator = ideal_generator(I2)

+ Compute the reduced basis $\{o_{11}, o_{12}, o_{13}, o_{14}\}$ of $\mathcal{O_1}$

In [137]:
O1_basis = GetReducedBasis(O1.basis())

+ Try to find the $\beta \in \mathcal{O}_1$ such that $n(\alpha_2) + n(\beta) = 2^{e_1}\cdot 3^{e_2} | \sqrt{\#E_1(\mathbb{F}_{p^k})}$
+ $\mathrm{ord}_{2}(\#E_1(\mathbb{F}_{p^k})) \approx \mathrm{ord}_2(\#E_1(\mathbb{F}_{p^2}))=2 \times \mathrm{ord}_2(p+1)$ regardless of the extension size $k$
    + So it must be satisfied that $2^e < (p+1)$
+ Find the coefficients $\{c_1, c_2, c_3, c_4\}$ such that $n(\beta) = n(\sum_{i}{c_i o_{1i}}) = 2^e - n(\alpha_2)$

In [138]:
import random
I2_basis = GetReducedBasis(I2.basis())
I2_generator = I2_basis[2]
e1 = 148
e2 = 16
gram = matrix([[1,0,0,0],[0,1,0,0],[0,0,prime,0],[0,0,0,prime]])
M_O1 = matrix([vector(v.coefficient_tuple()) for v in O1_basis])
G = M_O1 * gram * M_O1.transpose()

x,y = 0, 0
k = 100
for i in range(k^2):
    I2_gen = I2_generator + random.randint(-k, k) * I2_basis[1] + random.randint(-k, k) * I2_basis[0]
    if gcd(I2_gen.reduced_norm() ,6) != 1: continue
    target = 2^e1*3^e2 - I2_gen.reduced_norm()
    assert target > 0
    if kronecker(-G[1][1], target) != 1: continue
    try:
        x,y = cornacchia(G[1][1], target)
        break
    except:
        print("no solution of cornacchia algorithm")
beta = O1_basis[0]*x + O1_basis[1]*y
print((beta.reduced_norm() + I2_gen.reduced_norm()).factor())
assert gcd(beta.reduced_norm(), I2_gen.reduced_norm()) == 1

no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
2^148 * 3^16


In [139]:
N = 2^e1*3^e2
N_alpha = I2_gen.reduced_norm()
N_beta = beta.reduced_norm()
x_value = Integer(mod(N_beta, N)^-1)

In [140]:
P1, Q1 = get_basis(E1, N)

In [141]:
load('isogeny.sage')
psi_P1 = eval_endomorphism(x_value * I2_gen * beta, P1, N, phi1, phi1_dual)
psi_Q1 = eval_endomorphism(x_value * I2_gen * beta, Q1, N, phi1, phi1_dual)

In [142]:
points = [P1, Q1, psi_P1, psi_Q1]
points = [E1((e[0], e[1])) for e in points]
assert points[0].weil_pairing(points[1], N)*points[2].weil_pairing(points[3], N) == 1
ell_P, ell_Q = get_basis(E1, ell)
points = points + [ell_P, ell_Q]


In [143]:
load('../01. jacobian_isogeny/22_isogeny.sage')
points = [(points[0], points[2]), (points[1], points[3]), (points[4], E1(0)), (points[5], E1(0))]
a, b = 148, 16
kernel2 = [(3^b*2^(a-1)*D[0], 3^b*2^(a-1)*D[1]) for D in [points[0], points[1]]]
E_start = E1
h, points, isog = Gluing22(E_start, E_start, kernel2, eval_points = points)

In [144]:
kernel22 = [3^b * 2 * D for D in points[:2]]
j, points = isogeny_22(kernel22, points, a-2)

In [145]:
load('../01. jacobian_isogeny/33_isogeny.sage')
kernel33 = [2 * D for D in points[:2]]
j, points = isogeny_33(kernel33, points, b)

In [148]:
G1 = points[0][0]
G2 = points[1][0]
h = j.curve().hyperelliptic_polynomials()[0]
G3, r3 = h.quo_rem(G1*G2)
assert r3 == 0
delta = Matrix(G.padded_list(3) for G in (G1, G2, G3))
assert delta.determinant() == 0

AssertionError: 

In [ ]:
assert False

In [56]:
print(I2_gen)
print(beta)

135 - 6549/10*QA_i + 1/10*QA_j
-54763607599075260554044879 + 111160242456670126234994180*QA_i


In [66]:
j.curve().igusa_clebsch_invariants()

(295550081812067506920870491782768281339194366411316*Fp2_i^2 + 135930209919477404775676110846599292076667463228115,
 129769651395485917865724588999096132742386934485229*Fp2_i^2 + 154994434040007412778856560149495833719737474797745,
 150631864079125256976677285439094763899254131714405*Fp2_i^2 + 61843435630769455440589635675941384710415284706267,
 344411903338477214773442479064997693314506680484907*Fp2_i^2 + 189804423517325027364452134962714775443322862532290)

In [125]:
import random
load('isogeny.sage')

for case in range(20):
    I2_basis = GetReducedBasis(I2.basis())
    I2_generator = I2_basis[2]
    e1 = 148
    e2 = 16
    gram = matrix([[1,0,0,0],[0,1,0,0],[0,0,prime,0],[0,0,0,prime]])
    M_O1 = matrix([vector(v.coefficient_tuple()) for v in O1_basis])
    G = M_O1 * gram * M_O1.transpose()

    x,y = 0, 0
    k = 100
    for i in range(k^2):
        I2_gen = I2_generator + random.randint(-k, k) * I2_basis[1] + random.randint(-k, k) * I2_basis[0]
        if gcd(I2_gen.reduced_norm() ,6) != 1: continue
        target = 2^e1*3^e2 - I2_gen.reduced_norm()
        assert target > 0
        if kronecker(-G[1][1], target) != 1: continue
        try:
            x,y = cornacchia(G[1][1], target)
            break
        except:
            continue
    beta = O1_basis[0]*x + O1_basis[1]*y
    assert beta.reduced_norm() + I2_gen.reduced_norm() == 2^e1*3^e2
    assert gcd(beta.reduced_norm(), I2_gen.reduced_norm()) == 1
    print(I2_gen)
    print(I2_gen.reduced_norm().factor())
    print(beta)
    print(beta.reduced_norm().factor())

    N = 2^e1*3^e2
    N_alpha = I2_gen.reduced_norm()
    N_beta = beta.reduced_norm()
    x_value = Integer(mod(N_beta, N)^-1)

    P1, Q1 = get_basis(E1, N)

    psi_P1 = eval_endomorphism(x_value * I2_gen * beta, P1, N, phi1, phi1_dual)
    psi_Q1 = eval_endomorphism(x_value * I2_gen * beta, Q1, N, phi1, phi1_dual)

    points = [P1, Q1, psi_P1, psi_Q1]
    points = [E1((e[0], e[1])) for e in points]
    assert points[0].weil_pairing(points[1], N)*points[2].weil_pairing(points[3], N) == 1
    ell_P, ell_Q = get_basis(E1, ell)
    points = points + [ell_P, ell_Q]

    load('../01. jacobian_isogeny/22_isogeny.sage')
    points = [(points[0], points[2]), (points[1], points[3]), (points[4], E1(0)), (points[5], E1(0))]
    a, b = 148, 16
    kernel2 = [(3^b*2^(a-1)*D[0], 3^b*2^(a-1)*D[1]) for D in [points[0], points[1]]]
    E_start = E1
    h, points, isog = Gluing22(E_start, E_start, kernel2, eval_points = points)
    print(h.clebsch_invariants())

    kernel22 = [3^b * 2 * D for D in points[:2]]
    j, points = isogeny_22(kernel22, points, a-2)
    print(j.curve().clebsch_invariants())

    load('../01. jacobian_isogeny/33_isogeny.sage')
    kernel33 = [2 * D for D in points[:2]]
    j, points = isogeny_33(kernel33, points, b)
    print(j.curve().clebsch_invariants())

    G1 = points[0][0]
    G2 = points[1][0]
    h = j.curve().hyperelliptic_polynomials()[0]
    G3, r3 = h.quo_rem(G1*G2)
    assert r3 == 0
    delta = Matrix(G.padded_list(3) for G in (G1, G2, G3))
    if delta.determinant() == 0 : print("%d : success"%(case))
    else : print("%d : failed"%(case))

-401 + 20349/10*QA_i + 1/10*QA_j
5 * 83 * 283 * 5939 * 1636039 * 1286035343 * 2678663431797211707061061
-53908706130626607425169246 - 111577343684158938473261155*QA_i
2904353 * 24842669 * 212824006382171253710833987915372624513
(215465122963081338079170998413914965186873585759808, 222274173020397153114925510086177691512657171678984, 18874286550873828225542581397396296004480000465127, 100005866440512162766331626278440158016079412487043)
(356144630195844230672012334668396175977027930213581*Fp2_i^2 + 318573190768804151555740364551845119007235910062409, 38353029037591133225284283929636709410881995640243*Fp2_i^2 + 30392853772533309265538936355713874811912894092475, 355243432540431447087811044631516257442948111490810*Fp2_i^2 + 112487684743890612326877230696571510720525702976946, 377824290628585688666814712793784473387835831857113*Fp2_i^2 + 62928360324152631618912416905952538316682032892258)


TypeError: unsupported operand parent(s) for *: 'Finite Field in Fp2_i of size 393108616061618412489735027422936257282820381081599^4' and 'Rational Field'

In [ ]:
load('../01. jacobian_isogeny/22_isogeny.sage')
kernel22 = [D for D in points[:2]]
prodE, eval_points = Splitting22(kernel22, [points[2], points[3]])

In [ ]:
print(eval_points)

[((217681384236001118232143045409478000596796809013471*Fp2_i^2 + 213052435544866963096056658728022462815072557879709 : 217575137782513305318290947178346955315997005146879*Fp2_i^3 + 354512674135835281378964823815002048206768340628496*Fp2_i : 1), (157271264800523852986476289815525272233912789229822*Fp2_i^2 + 56880086213987454445252619131375661647846608821902 : 144777345766818464557002921507128515193559612507604*Fp2_i^3 + 249949666982062834605542624331672405197171330592696*Fp2_i : 1)), ((366001795385812304401668034514116123212454592161177*Fp2_i^2 + 268360151052555436165135994966191533776122252746445 : 354151999919731032642549544607893411255953191331973*Fp2_i^3 + 309251391409297574738728199143362245425129164853364*Fp2_i : 1), (341999281521453162741513911253593759989459549463812*Fp2_i^2 + 191476377378440474857750183759707383215810818261188 : 211765567622185912025873313411961799971766465496595*Fp2_i^3 + 7811662083241675113916649488081804563423427885595*Fp2_i : 1))]


In [ ]:
assert E1.j_invariant() in [T.j_invariant() for T in prodE]
P, Q = E1(0), E1(0)
if prodE[0].j_invariant() == E1.j_invariant():
    P, Q = eval_points[0][0], eval_points[1][0]
else:
    P, Q = eval_points[0][1], eval_points[1][1]

second_kernel = E1(0)
if P.is_zero(): second_kernel = ell_Q
elif Q.is_zero(): second_kernel = ell_P
else:
    assert P.weil_pairing(Q, P.order()) == 1

    r = P.discrete_log(Q)
    second_kernel = ell_P * r - ell_Q
print("second kernel :", second_kernel)


second kernel : (190109187736188721217378289285843068400227307187967*Fp2_i^2 + 215889668914670661327335186988343309364958840599297 : 326102712949964461182852671185483986295906132819430*Fp2_i^2 + 335029515558269756534411781187261354934571241310845 : 1)


In [ ]:
O2 = I2.right_order()

In [ ]:
I3 = I2.conjugate() * I1.conjugate() * I
I3 = I3.scale(ell^-2)
ellO2 = QA.ideal(O2.basis()).scale(ell)
I3 = AddIdeal(I3, ellO2)
assert I3.norm() == ell and I3.left_order() == O2

In [ ]:
load('isogeny.sage')
phi2 = Isogeny(phi1.codomain_coeff, second_kernel, 2^e1*3^e2*ell)

In [ ]:
load('isogeny.sage')
E2 = phi2.codomain_curve
phi2_dual = phi2.dual_isogeny()

In [ ]:
cofactor = phi2_dual.domain_P.order() // 3^16
T = cofactor * phi2_dual.codomain_P
ellT = phi2.eval(T)
if ellT != phi2.degree * cofactor * phi2_dual.domain_P:
    phi2_dual.codomain_P *= -1
    phi2_dual.codomain_Q *= -1
    assert -ellT == phi2.degree * cofactor * phi2_dual.domain_P
else: assert ellT == phi2.degree * cofactor * phi2_dual.domain_P

T = cofactor * phi2.codomain_P
ellT = phi2_dual.eval(T)
assert ellT == phi2_dual.degree * cofactor * phi2.domain_P

In [ ]:
I3_generator = ideal_generator(I3)

In [ ]:
O2_basis = GetReducedBasis(O2.basis())

### Computing $\beta$ such that $n(\beta) + n(\alpha_3) = 2^{e_1}\times 3^{e_2}$

In [ ]:
import random
I3_basis = GetReducedBasis(I3.basis())
I3_generator = I3_basis[2]
e1 = 148
e2 = 16
gram = matrix([[1,0,0,0],[0,1,0,0],[0,0,prime,0],[0,0,0,prime]])
M_O2 = matrix([vector(v.coefficient_tuple()) for v in O2_basis])
G = M_O2 * gram * M_O2.transpose()

x,y = 0, 0
k = 100
for i in range(k^2):
    I3_gen = I3_generator + random.randint(-k, k) * I3_basis[1] + random.randint(-k, k) * I3_basis[0]
    if gcd(I3_gen.reduced_norm() ,6) != 1: continue
    target = 2^e1*3^e2 - I3_gen.reduced_norm()
    assert target > 0
    if kronecker(-G[1][1], target) != 1: continue
    try:
        x,y = cornacchia(G[1][1], target)
        break
    except:
        continue
beta = O2_basis[0]*x + O2_basis[1]*y
assert (beta.reduced_norm() + I3_gen.reduced_norm()) == 2^e1*3^e2
assert gcd(beta.reduced_norm(), I3_gen.reduced_norm()) == 1

### Compute $x \equiv \beta^{-1} \mod N$ 

In [ ]:
N = 2^e1*3^e2
N_alpha = I3_gen.reduced_norm()
N_beta = beta.reduced_norm()
x_value = Integer(mod(N_beta, N)^-1)

In [ ]:
P1, Q1 = get_basis(phi2.codomain_curve, N)
E2.is_on_curve(P1[0], P1[1])

True

In [ ]:
load('isogeny.sage')
isog = IsogenyChain([phi1, phi2])
isog_dual = IsogenyChain([phi2_dual, phi1_dual])
psi_P1 = eval_endomorphism(x_value * I3_gen * beta, P1, N, isog, isog_dual)
psi_Q1 = eval_endomorphism(x_value * I3_gen * beta, Q1, N, isog, isog_dual)

#### Check if $e_N((P, \phi(P)), (Q, \phi(Q))) = e_N(P, Q)\cdot e_N(\phi(P), \phi(Q))$
+ where $\phi = [x] \circ \theta_{\alpha} \circ \theta_{\beta}$
+ $\theta_{\alpha}$ and $\theta_{\beta}$ are separable endomorphisms

In [ ]:
points = [P1, Q1, psi_P1, psi_Q1]
points = [E2((e[0], e[1])) for e in points]
assert points[0].weil_pairing(points[1], N)*points[2].weil_pairing(points[3], N) == 1
[ell_P, ell_Q] = get_basis(E2, ell)
points = points + [ell_P, ell_Q]

### Computing Isogeny in hyperelliptic curves

In [ ]:
load('../01. jacobian_isogeny/22_isogeny.sage')
points = [(points[0], points[2]), (points[1], points[3]), (points[4], E1(0)), (points[5], E1(0))]
a, b = 148, 16
kernel2 = [(3^b*2^(a-1)*D[0], 3^b*2^(a-1)*D[1]) for D in [points[0], points[1]]]
E_start = E2
h, points, isog = Gluing22(E_start, E_start, kernel2, eval_points = points)

In [ ]:
kernel22 = [3^b * 2 * D for D in points[:2]]
j, points = isogeny_22(kernel22, points, a-2)

In [ ]:
load('../01. jacobian_isogeny/33_isogeny.sage')
kernel33 = [2 * D for D in points[:2]]
K = Fp
j, points = isogeny_33(kernel33, points, b)

Interrupting Singular...


KeyboardInterrupt: Restarting Singular (WARNING: all variables defined in previous session are now invalid)

In [ ]:
G1 = points[0][0]
G2 = points[1][0]
h = j.curve().hyperelliptic_polynomials()[0]
G3, r3 = h.quo_rem(G1*G2)
assert r3 == 0
delta = Matrix(G.padded_list(3) for G in (G1, G2, G3))
assert delta.determinant() == 0

AssertionError: 

In [ ]:
print(I3_gen.reduced_norm().factor())
print(beta.reduced_norm().factor())

In [ ]:
print(I3_gen)